In [1]:
from imutils.video import VideoStream
from imutils.video import FPS
from IPython.display import clear_output
import argparse
import imutils
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [39]:
cap = cv2.VideoCapture('video1.mp4')

In [40]:
(major, minor) = cv2.__version__.split(".")[:2]
print(major, minor)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)
!rm -f face_tracker.avi

result = cv2.VideoWriter('face_tracker.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

4 4


In [41]:
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}

tracker_method = "mosse"

tracker = OPENCV_OBJECT_TRACKERS[tracker_method]()

fps = None
initBB = None

In [42]:
first_run = True
frame_no = 0
while True:
	# grab the current frame, then handle if we are using a
	# VideoStream or VideoCapture object
    ret, frame = cap.read()
    if ret is False:
        break
    
    print(frame_no)
    
    (H, W) = frame.shape[:2]
    
    # check to see if we are currently tracking an object
    if initBB is not None:
		# grab the new bounding box coordinates of the object
        (success, box) = tracker.update(frame)
		# check to see if the tracking was a success
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
		# update the FPS counter
        fps.update()
        fps.stop()
		# initialize the set of information we'll be displaying on
		# the frame
        info = [
			("Tracker", tracker_method),
			("Success", "Yes" if success else "No"),
			("FPS", "{:.2f}".format(fps.fps())),
        ]
		# loop over the info tuples and draw them on our frame
        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
    # show the output frame
    result.write(frame)
    
	# if the 's' key is selected, we are going to "select" a bounding
	# box to track
    if first_run:
		# select the bounding box of the object we want to track (make
		# sure you press ENTER or SPACE after selecting the ROI)
        initBB = (1265, 188, 44, 50)
		# start OpenCV object tracker using the supplied bounding box
		# coordinates, then start the FPS throughput estimator as well
        tracker.init(frame, initBB)
        fps = FPS().start()
    frame_no += 1
    clear_output(wait = True)
cap.release()

1775


KeyboardInterrupt: 